In [15]:
#Import standard packages
import pandas as pd
import numpy as np
import xarray as xr
import math
import os

from pyaldata import *

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import io
from scipy import stats
from sklearn.metrics import r2_score
import pickle
from tqdm import tqdm
import csv

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import decoder functions
from Neural_Decoding.decoders import WienerFilterDecoder

In [3]:
folder='../preprocessed_data/'
#ENTER THE FOLDER THAT YOUR DATA IS IN

with open(folder+'all_data.pickle','rb') as f:
    M1, PMd, M1_PMd,pos_binned,vels_binned=pickle.load(f,encoding='latin1') #If using python 3


In [8]:
# Function that reassigns different angles into classes from 1 to 8
# going anticlockwise starting from +x direction
def determine_angle(angle):
    if angle == 0:
        return 1
    elif angle == math.pi/4:
        return 2
    elif angle == math.pi/2:
        return 3
    elif angle == 3*math.pi/4:
        return 4
    elif angle == math.pi:
        return 5
    elif angle == -3*math.pi/4:
        return 6
    elif angle == -math.pi/2:
        return 7
    elif angle == -math.pi/4:
        return 8

In [32]:
# Load data
import os
data_dir = '../raw_data/'
fname = os.path.join(data_dir, "Chewie_CO_CS_2016-10-14.mat")

# load TrialData .mat file into a DataFrame
df = mat2dataframe(fname, shift_idx_fields=True)

# Keep only successful trials
df = select_trials(df, "result == 'R'")

# Import the whole data set
# combine time bins into longer ones
td_full = combine_time_bins(df, 3)

# Remove low-firing neurons
td_full = remove_low_firing_neurons(td_full, "M1_spikes",  5)
td_full = remove_low_firing_neurons(td_full, "PMd_spikes", 5)

# Get the signal from idx_go_cue
df.idx_movement_on = df.idx_movement_on.astype(int)
td_full = restrict_to_interval(td_full, start_point_name='idx_go_cue', end_point_name='idx_trial_end')


td_full = smooth_signals(td_full, ["M1_spikes", "PMd_spikes"], std=0.05)

# Get the data in the right format
N_full = td_full.shape[0]

lengths = []
angle = []
for i in range(N_full):
    # get position
    pos = td_full.pos[i]
    lengths.append(pos.shape[0])
    angle.append(determine_angle(td_full.target_direction[i]))

In [39]:
angle_data = np.zeros(shape=(M1.shape[0],1))
start = 0
for (idx,duration) in enumerate(lengths):
    end = start + duration
    angle_data[start:end] = angle[idx]
    
    
    start = end


print(M1.shape, angle_data.shape)
print(np.expand_dims(angle_train.shape, axis=1))

(30068, 63) (30068, 1)
[[21042]]


In [47]:
## Do preprocessing and run the decoder for M1, PMd, and M1&PMd together 
#neural_data_combined = [M1, PMd, M1_PMd]
neural_data_combined = [M1]
kinematics = [pos_binned, vels_binned]

R2 = np.zeros((3,2))
row = 0
for i in tqdm(range(1)):
    neural_data = neural_data_combined[i]
    col = 0
    #print('Neural data:', neural_data.shape)
    for output in kinematics:
        # Preprocess data
        bins_before=6 #How many bins of neural data prior to the output are used for decoding
        bins_current=1 #Whether to use concurrent time bin of neural data
        bins_after=0 #How many bins of neural data after the output are used for decoding

        # Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
        # Function to get the covariate matrix that includes spike history from previous bins
        X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

        # Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
        #Put in "flat" format, so each "neuron / time" is a single feature
        X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

        # Output covariates
        #Set decoding output
        y=output
        
        # Split into training / testing / validation sets
        #Set what part of data should be part of the training/testing/validation sets
        training_range=[0, 0.7]
        testing_range=[0.7, 0.85]
        valid_range=[0.85,1]

        # Split data:
        num_examples=X.shape[0]

        #Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
        #This makes it so that the different sets don't include overlapping neural data
        training_set=np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
        testing_set=np.arange(int(np.round(testing_range[0]*num_examples))+bins_before,int(np.round(testing_range[1]*num_examples))-bins_after)
        valid_set=np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)
        
        angle_train = np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
        angle_valid = np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)
        angle_train = np.expand_dims(angle_train, axis=1)
        angle_valid = np.expand_dims(angle_valid, axis=1)
        
        #Get training data
        X_train=X[training_set,:,:]
        X_flat_train=X_flat[training_set,:]
        y_train=y[training_set,:]

        #Get testing data
        X_test=X[testing_set,:,:]
        X_flat_test=X_flat[testing_set,:]
        y_test=y[testing_set,:]

        #Get validation data
        X_valid=X[valid_set,:,:]
        X_flat_valid=X_flat[valid_set,:]
        y_valid=y[valid_set,:]
        
        # Process covariates
        #Z-score "X" inputs. 
        X_train_mean=np.nanmean(X_train,axis=0)
        X_train_std=np.nanstd(X_train,axis=0)
        X_train=(X_train-X_train_mean)/X_train_std
        X_test=(X_test-X_train_mean)/X_train_std
        X_valid=(X_valid-X_train_mean)/X_train_std

        #Z-score "X_flat" inputs. 
        X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
        X_flat_train_std=np.nanstd(X_flat_train,axis=0)
        X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
        X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
        X_flat_valid=(X_flat_valid-X_flat_train_mean)/X_flat_train_std
        
        # Add the angle
        X_flat_train = np.hstack((X_flat_train, angle_train))
        X_flat_valid = np.hstack((X_flat_valid, angle_valid))
        #Zero-center outputs
        y_train_mean=np.mean(y_train,axis=0)
        y_train=y_train-y_train_mean
        y_test=y_test-y_train_mean
        y_valid=y_valid-y_train_mean
        
        # Run decoder
        #Declare model
        model_wf=WienerFilterDecoder()
        
        #print(X_flat_train.shape)
        #Fit model
        model_wf.fit(X_flat_train,y_train)

        #Get predictions
        y_valid_predicted_wf=model_wf.predict(X_flat_valid)
        #print(X_flat_valid.shape, y_valid_predicted_wf.shape)
        
        R2_vw = r2_score(y_valid,y_valid_predicted_wf, multioutput='variance_weighted')
        
        # Save the R2 value for a given neural data and kinematics
        R2[row,col] = R2_vw
        col += 1
    row += 1

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.19s/it]


In [4]:
# Export the data into a csv file for further analysis
csv_rowlist = [['','Wiener Filter'], ['R^2 values','Position', 'Velocity'], ['M1', R2[0,0], R2[0,1]], ['PMd',  R2[1,0], R2[1,1]],['M1 & PMd', R2[2,0], R2[2,1]]]

export_path = '../Results/WF_raw.csv'
with open(export_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_rowlist)

In [5]:
print(R2)

[[0.7901467  0.82098066]
 [0.73142784 0.79040175]
 [0.86916823 0.88443858]]


In [48]:
print(R2)

[[0.80410819 0.81457269]
 [0.         0.        ]
 [0.         0.        ]]
